In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from pathlib import Path
import itertools 

from cycler import cycler

import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import matplotlib.cm as mcm
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

from matplotlib import rcParams

from scipy.spatial.distance import pdist, squareform

import numpy as np
import pandas as pd
import cmocean
import xarray as xr

import metpy
from metpy.interpolate import interpolate_to_grid

import geopandas as gpd


import intro_plots as ip
from papercolors import colordict as pcd
import topology as tp 

paper_path = Path('../figures/')


rcParams['font.family'] = 'dejavu sans'
rcParams['figure.dpi'] = 200

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

In [ ]:
fig, ax = plt.subplots(figsize=(5,1))
ax.bar(ip.airports.keys(), 1, color=[v['color'] for k,v in ip.airports.items()]) 
ax.tick_params(rotation=90)

In [ ]:
ny_all = pd.read_csv("nydata.csv", parse_dates=['DATE'])
#[name for name in ny_all['NAME'].unique() if (('AP' in name) or ('INTL' in name))]

In [ ]:
ny = ny_all[ny_all['NAME'].isin(ip.ssubset)].copy().fillna(0)

ny['DATES'] = pd.to_datetime(ny['DATE'])

ny['TEMP (°F)'] = ny['TAVG'].apply(lambda x: f"{x:2.2f}")
ny['PRCP (in.)'] = ny['PRCPI'].apply(lambda x: f"{x:1.2f}")    

In [ ]:
ny['NAME'].unique()

In [ ]:
date = '02-10-2022'
time_series = ny.copy()
time_series['DATE'] = pd.to_datetime(time_series['DATES'])
all_fields = ny[ny['DATE'] == date].set_index('NAME').reset_index()
all_fields

In [ ]:
nyg = ny_all[ny_all['DATE']==date]
nygeo = gpd.GeoDataFrame(nyg, geometry=gpd.points_from_xy(nyg['LONGITUDE'], nyg['LATITUDE']), crs='EPSG:4269')
gdf = gpd.read_file('s_22mr22.zip')
nyshp = gdf[gdf['STATE'].str.match('NY')]

In [ ]:
px, py, prcp = interpolate_to_grid(nygeo['LONGITUDE'].values, nygeo['LATITUDE'].values, nygeo['PRCPI'].values, 
                                   interp_type='barnes', minimum_neighbors=3, 
                                         search_radius=.25, hres=.025, gamma=1)

In [ ]:
sample = all_fields[['NAME', 'TEMP (°F)','PRCP (in.)']]

In [ ]:
timeseries = {k:v.set_index('DATE') for k, v in time_series[['NAME','DATE', 'PRCPI']].groupby('NAME')}

In [ ]:
all_fields[['NAME', 'LATITUDE', 'LONGITUDE', 'TAVG']]

In [ ]:
mcolors.to_hex(cmocean.cm.rain(0))

In [ ]:
fig = plt.figure(figsize=(3.5, 1.5), linewidth=1, layout='compressed')
figs = fig.subfigures(1,4, wspace=0)
for subfig in figs:
    #fig0.frameon=True
    subfig.set_edgecolor('gray')
    subfig.set_linewidth(.5)


axf0 = figs[1].subplot_mosaic([['scatter'], ['k0']], gridspec_kw={'height_ratios':[2,1]})
#tp.make_table(axf0['table'], axf0['k0'], sample)

axf1 = figs[2].subplot_mosaic([['timeseries'],['k1']], gridspec_kw={'height_ratios':[2,1]})
#tp.make_timeseries(axf1['timeseries'], axf1['k1'], timeseries)
axf1['timeseries'].set_title('Precipitation', fontsize=6)
axf1['timeseries'].tick_params(labelsize=3, length=0
                              )
axf1['timeseries'].xaxis.set_tick_params(rotation=90)
axf1['timeseries'].yaxis.set_major_formatter("{:0.1f}".format)

axf2 = figs[3].subplot_mosaic([['map'], ['k2']], gridspec_kw={'height_ratios':[2,1]})

cmap = cmocean.cm.rain
nyshp.plot(ax=axf2['map'],  facecolor='#f6f5f7', edgecolor='none')
im = axf2['map'].pcolormesh(px, py, prcp, cmap=cmap)
nyshp.plot(ax=axf2['map'],  facecolor='none', edgecolor='k')
cb = fig.colorbar(im, ax=axf2['map'], fraction=.045,orientation='horizontal',
                  location='bottom')
#axf2['map'].set_title("Precipitation", fontsize=6)
cb.ax.tick_params(labelsize=3, length=0, pad=0.5,rotation=90)
cb.ax.xaxis.set_major_formatter("{:0.2f} in.".format)
axf2['map'].axis('off')
           
           
axf2['k2'].add_patch(mpatches.Rectangle((.25/2,.25-.25/2), .75, .75, color=pcd['base']))
axf2['k2'].set_aspect('equal')
axf2['k2'].axis('off')
fig.savefig(paper_path/'k_different_types.png')

In [ ]:
cb.ax.tick_params?